# <font color="orange">Create Solvated NP Structures</font>
Patrick Butler - Jan 2025

### Contents
-	[create coreshell NP structures](#section-1)
-   [add ligands to NP](#section-2)
-   [create solvent boxes](#section-3)
-   [add NPs to solvent box](#section-4)
-   [add solvent shell to NPs](#section-5)

In [65]:
from simulation_cell_utils import *
import numpy as np
from ase.io import read
from ase import Atom, Atoms
import random
from ase.visualize import view
from ase.build import molecule

## <font color="red">Create NPs</font> <a id="section-1"></a>

In [30]:
from ase.visualize import view
from ase.lattice.cubic import FaceCenteredCubic, SimpleCubic, BodyCenteredCubic
from ase.lattice.hexagonal import HexagonalClosedPacked
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.decahedron import Decahedron
from ase.cluster.octahedron import Octahedron


In [69]:
def recursive_magic_numbers(n):
    """get the number of atoms in a Mackay Icosahedron with n shells"""
    assert n >= 0, "n must be positive"
    if n == 0:
        return 1
    else:
        prev_layers = recursive_magic_numbers(n-1)
        return prev_layers + (10*n**2 + 2)

def create_core_shell_icosahedral(nshells, core_atom_type, shell_atom_type, lattice_constant=None):
    """
    NOTE: if lattice constant None will use shell atom type lattice constant
    """
    num_core_atoms = recursive_magic_numbers(nshells-2)
    cluster = Icosahedron(shell_atom_type, nshells, lattice_constant)

    for atom in cluster[:num_core_atoms]:
        cluster[atom.index].symbol = core_atom_type

    return cluster

In [33]:
lc = 2.72 # mean of Cu and Au bulk https://pubs.acs.org/doi/10.1021/nl5005674
atoms = create_core_shell_icosahedral(4, "Cu", "Au", 2*np.sqrt(0.5*lc**2)) # something from Andy's code that looks better
view(atoms, viewer="x3d")

In [34]:
atoms.get_chemical_formula()

'Au92Cu55'

## <font color="red">Add Ligands to Nanoparticle</font> <a id="section-2"></a>

Adding ligands can be done with ACAT which is demonstrated below but requires installing ACAT and changing the settings file to add absorbates not included in ACAT

### Using ACAT

see the documentation (https://asm-dtu.gitlab.io/acat/) and example notebook (https://gitlab.com/asm-dtu/acat/-/blob/master/notebooks/acat_tutorial.ipynb)

In acat/acat/settings.py add the following:

```python
monodentate_adsorbate_list = ['H','C','N','O','S',
                              'CH','NH','OH','SH','CO','NO','CN','CS','NS',
                              'CH2','NH2','OH2','SH2','COH','NOH',
                              'CH3','NH3','OCH',"CH3CH2OH",
                              'OCH2',
                              'OCH3',]
def adsorbate_molecule(adsorbate):
    ...
    elif adsorbate == "CH3CH2OH":
        ads = molecule("CH3CH2OH")
        # ads = ads[0,6,7,8,1,4,5,2,3] # terminal C (note the hydrogens follow the atom)
        ads = ads[2,3,0,6,7,8,1,4,5,] # terminal 
        ads.rotate(-90, 'y') # the attached atom should be along the negative z directions (I think). Just use the ase viewer and try different rotations and add to NP until get it right
```

In [59]:
ethanol = molecule("CH3CH2OH")
ethanol.rotate(-90, 'y')
view(ethanol, viewer="x3d")

In [66]:
from acat.build.adlayer import RandomPatternGenerator as RPG
from acat.build.adlayer import max_dist_coverage_pattern as maxdcp
from acat.adsorption_sites import ClusterAdsorptionSites


# Initialize a pure Ag Dh nanoparticle
proto = Decahedron('Ag', p=2, q=2, r=1)
# First identify the adsorption sites to speed up the generation
cas_dh = ClusterAdsorptionSites(proto, allow_6fold=False, composition_effect=False, ignore_sites=["hcp", "fcc", "bridge", "4fold"])
# Generate random adsorbate coverage patterns
rpg = RPG(proto, adsorbate_species=["CH3CH2OH"],
                  min_adsorbate_distance=2.,
                  adsorption_sites=cas_dh,
                  trajectory='patterns_dh.traj',heights={"ontop": 3.})
rpg.run(num_gen=1, action='add', num_act=8)
# # Visualize the output structures
structure_with_ligands = read('patterns_dh.traj', index='-1')
view(structure_with_ligands, viewer="x3d")

# pattern = maxdcp(proto, adsorbate_species='CO', coverage=0.67, adsorption_sites=cas_dh, ) # requires installing pyclustering
#view(pattern, viewer='x3d')

# pattern.write("pattern.xyz")

## <font color="red">Create Solvent Box</font> <a id="section-3"></a>

#### Equilibrating a TIPnP Water Box
https://wiki.fysik.dtu.dk/ase/tutorials/tipnp_equil/tipnp_equil.html

In [66]:
import ase.units as units
from ase import Atoms
from ase.calculators.tip3p import TIP3P, angleHOH, rOH
from ase.constraints import FixBondLengths
from ase.io.trajectory import Trajectory
from ase.md import Langevin
from ase.visualize import view


# Set up water box at 20 deg C density
x = angleHOH * np.pi / 180 / 2
pos = [[0, 0, 0],
       [0, rOH * np.cos(x), rOH * np.sin(x)],
       [0, rOH * np.cos(x), -rOH * np.sin(x)]]
atoms = Atoms('OH2', positions=pos)

vol = ((18.01528 / 6.022140857e23) / (0.9982 / 1e24))**(1 / 3.)
atoms.set_cell((vol, vol, vol))
atoms.center()

atoms = atoms.repeat((3, 3, 3)) # make some multiple smaller than the actual size
atoms.set_pbc(True)

# RATTLE-type constraints on O-H1, O-H2, H1-H2.
atoms.constraints = FixBondLengths([(3 * i + j, 3 * i + (j + 1) % 3)
                                    for i in range(3**3)
                                    for j in [0, 1, 2]])


tag = 'tip3p_27mol_equil'
atoms.calc = TIP3P(rc=4.5)
md = Langevin(atoms, 1 * units.fs, temperature=300 * units.kB,
              friction=0.01, logfile=tag + '.log')

# traj = Trajectory(tag + '.traj', 'w', atoms)
# md.attach(traj.write, interval=1)

# md.run(4000)

# Repeat box and equilibrate further.
tag = 'tip3p_216mol_equil'
atoms.set_constraint()  # repeat not compatible with FixBondLengths currently.
atoms = atoms.repeat((2, 2, 2))


atoms.constraints = FixBondLengths([(3 * i + j, 3 * i + (j + 1) % 3)
                                    for i in range(len(atoms) // 3)
                                    for j in [0, 1, 2]])


atoms.calc = TIP3P(rc=7.)
md = Langevin(atoms, 2 * units.fs, temperature=300 * units.kB,
              friction=0.01, logfile=tag + '.log')

# traj = Trajectory(tag + '.traj', 'w', atoms)
# md.attach(traj.write, interval=1)

# md.run(2000)

# atoms.write("tip3p_216mol_equil_300K.xyz", format="extxyz")

#### Equilibrating an ethanol box at 298 K

In [8]:
# Set up water box at 20 deg C density
atoms = molecule("CH3CH2OH")

vol = ((46.068 / 6.022140857e23) / (0.7892 / 1e24))**(1 / 3.) # density of ethanol at 298 K = 0.7892 g / cm^3
atoms.set_cell((vol, vol, vol))
atoms.center()
atoms.rotate(random.uniform(0, 360), 'x')
atoms.rotate(random.uniform(0, 360), 'y')
atoms.rotate(random.uniform(0, 360), 'z')

atoms = atoms.repeat((3, 3, 3))
atoms.set_pbc(True)
# atoms.wrap()

Use some potential and run similar MD to the one for water to equilibrate the ethanol

#### Create ethanol-water mixture

In [53]:
# Load the initial structure containing water molecules
atoms = read('solvent_boxes/tip3p_216mol_equil_300K.xyz')
N = 7  # Number of ethanol molecules to add for 10% v/v EtOH (molar/molecule percentage = 0.0343 EtOH, and 216 * 0.0343 ~= 7)
ethanol = molecule("CH3CH2OH")  # Load the ethanol molecule

In [54]:
atoms = substitute_n_water_molecules(atoms, ethanol, N)

placing mol 1 at [ 5.46677551  3.60056753 14.88805275]
placing mol 2 at [16.44329498  8.75003281 11.66971291]
placing mol 3 at [-3.14983723 16.40087718  8.87605181]
placing mol 4 at [ 9.11225162  6.69754441 -2.60006301]
clash with added atoms (smallest dist 0.9483364812630107), trying again
placing mol 4 at [20.27578503 12.22458671 -0.30993183]
placing mol 5 at [15.30374168  3.34060859  2.8967624 ]
placing mol 6 at [ 1.01876937 23.10383123 12.72923445]
clash with added atoms (smallest dist 0.7811429068953294), trying again
placing mol 6 at [ 7.19808027 15.60757676  5.90416944]
placing mol 7 at [ 8.9530173  20.75643737 13.2597351 ]


In [57]:
# view(atoms, viewer="x3d")

In [58]:
atoms.wrap()
view(atoms, viewer="x3d")

In [14]:
# atoms.write("ethanol_water_10percent.xyz", format="extxyz")

## <font color="red">Add Nanoparticle to Solvent Box</font> <a id="section-4"></a>

In [44]:
solvent_box = read('solvent_boxes/tip3p_216mol_equil_300K.xyz')
np_atoms = create_core_shell_icosahedral(3, "Cu", "Au", 2*np.sqrt(0.5*lc**2))

In [49]:
simulation_cell = add_solute_to_solvent_box(solvent_box, np_atoms, solvent_box.get_center_of_mass(), solvent_n_atoms=3)
view(simulation_cell, viewer="x3d")

## <font color="red">Create Solvent Shell Structures</font> <a id="section-5"></a>

The function will put the nanoparticle (possilby with ligands attached) in a solvent box and remove all overlapping solvent moelcules as well as all solvent molecules beyond the radial cutoff, yielding a solvent shell

In [67]:
water_box = read("solvent_boxes/tip3p_216mol_equil_300K.xyz").repeat((2,2,2))
midpoint = water_box.cell.cellpar()[:3] / 2

atoms =create_solvent_shell_structure(water_box, structure_with_ligands, np_position=midpoint, solvent_n_atoms=3, radial_cutoff=12.0)

In [68]:
view(atoms, viewer="x3d")

Can further replace waters in the solvent shell

In [71]:
na_atom = Atoms([Atom("Na")])
new_atoms = substitute_n_water_molecules(atoms, na_atom, 3)
cl_atom = Atoms([Atom("Cl")])
new_atoms = substitute_n_water_molecules(new_atoms, cl_atom, 3)

placing mol 1 at [ 4.14086586 -8.31742559 -6.90702713]
placing mol 2 at [ 3.74023769  9.11268501 -4.45460823]
placing mol 3 at [-6.27863087 -4.10213766  7.02507802]
clash with non water atoms
placing mol 3 at [3.75858491 7.58404122 6.61553755]
clash with non water atoms
placing mol 3 at [ 4.68554225  1.40061099 -8.51386091]
placing mol 1 at [3.4137702  2.10982601 5.52359621]
clash with non water atoms
placing mol 1 at [ 5.52359621 -3.4137702   2.10982601]
clash with non water atoms
placing mol 1 at [ 2.10982601 -5.52359621 -3.4137702 ]
clash with non water atoms
placing mol 1 at [ 3.20058891 -0.76852893  7.27025385]
placing mol 2 at [-1.01871762  9.26279464  7.06703639]
placing mol 3 at [-5.08805552 -8.70566235  1.09641602]


In [72]:
view(new_atoms, viewer="x3d")